## Q1. Install MLflow

To get started with MLflow you'll need to install the MLflow Python package.

For this we recommend creating a separate Python environment, for example, you can use conda environments, and then install the package there with pip or conda.

Once you installed the package, run the command mlflow --version and check the output.

What's the version that you have?

In [1]:
!mlflow --version

mlflow, version 2.22.0


## Q2. Download and preprocess the data
We'll use the Green Taxi Trip Records dataset to predict the duration of each trip.

Download the data for January, February and March 2023 in parquet format from here.

Use the script preprocess_data.py located in the folder homework to preprocess the data.

The script will:

- load the data from the folder <TAXI_DATA_FOLDER> (the folder where you have downloaded the data),
- fit a DictVectorizer on the training set (January 2023 data),
- save the preprocessed datasets and the DictVectorizer to disk.
Your task is to download the datasets and then execute this command:

```python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output```

Tip: go to 02-experiment-tracking/homework/ folder before executing the command and change the value of <TAXI_DATA_FOLDER> to the location where you saved the data.

How many files were saved to OUTPUT_FOLDER?

In [2]:
# Get the dataset

!wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet -P ./data
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet -P ./data
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-03.parquet -P ./data

--2025-05-23 21:13:59--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.164.82.197, 3.164.82.112, 3.164.82.160, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.164.82.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1427002 (1.4M) [binary/octet-stream]
Saving to: './data/green_tripdata_2023-01.parquet.10'

     0K .......... .......... .......... .......... ..........  3%  254K 5s
    50K .......... .......... .......... .......... ..........  7%  532K 4s
   100K .......... .......... .......... .......... .......... 10%  496K 3s
   150K .......... .......... .......... .......... .......... 14%  644K 3s
   200K .......... .......... .......... .......... .......... 17% 1.68M 2s
   250K .......... .......... .......... .......... .......... 21%  933K 2s
   300K .......... .......... .......... .......... .......... 2

In [3]:
!python preprocess_data.py --raw_data_path ./data --dest_path ./output

In [4]:
import os
os.listdir("./output")

['dv.pkl', 'test.pkl', 'train.pkl', 'val.pkl']

**4 Files** in output folder

## Q3. Train a model with autolog
We will train a RandomForestRegressor (from Scikit-Learn) on the taxi dataset.

We have prepared the training script train.py for this exercise, which can be also found in the folder homework.

The script will:

- load the datasets produced by the previous step,
- train the model on the training set,
- calculate the RMSE score on the validation set.
Your task is to modify the script to enable autologging with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked.

Tip 1: don't forget to wrap the training code with a with mlflow.start_run(): statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

In [5]:
!python train.py

🏃 View run bemused-boar-500 at: http://127.0.0.1:5000/#/experiments/510162088933463138/runs/21f111fc61a54176896d1d5b68e7c08b
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/510162088933463138


In [6]:
from mlflow.tracking import MlflowClient

TRACKING_URI =  "http://127.0.0.1:5000"

client = MlflowClient(tracking_uri=TRACKING_URI)

In [7]:
client.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/510162088933463138', creation_time=1748003263023, experiment_id='510162088933463138', last_update_time=1748003263023, lifecycle_stage='active', name='homework2', tags={}>]

In [8]:
client.search_runs(experiment_ids=510162088933463138)

[<Run: data=<RunData: metrics={'training_mean_absolute_error': 3.4244701942312354,
  'training_mean_squared_error': 27.083054499499358,
  'training_r2_score': 0.6673983775155525,
  'training_root_mean_squared_error': 5.204138209108148,
  'training_score': 0.6673983775155525}, params={'bootstrap': 'True',
  'ccp_alpha': '0.0',
  'criterion': 'squared_error',
  'max_depth': '10',
  'max_features': '1.0',
  'max_leaf_nodes': 'None',
  'max_samples': 'None',
  'min_impurity_decrease': '0.0',
  'min_samples_leaf': '1',
  'min_samples_split': '2',
  'min_weight_fraction_leaf': '0.0',
  'monotonic_cst': 'None',
  'n_estimators': '100',
  'n_jobs': 'None',
  'oob_score': 'False',
  'random_state': '0',
  'verbose': '0',
  'warm_start': 'False'}, tags={'estimator_class': 'sklearn.ensemble._forest.RandomForestRegressor',
  'estimator_name': 'RandomForestRegressor',
  'mlflow.log-model.history': '[{"run_id": "21f111fc61a54176896d1d5b68e7c08b", '
                              '"artifact_path": "mo

In [9]:
run_id = "0d915bd9a5714c14b5fbb9278312a7b1"

run = client.get_run(run_id=run_id)

In [10]:
run.data.params.get('min_samples_split', None)

'2'

## Q4. Launch the tracking server locally
Now we want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry.

Your task is to:

- launch the tracking server on your local machine,
- select a SQLite db for the backend store and a folder called `artifacts` for the artifacts store.

You should keep the tracking server running to work on the next two exercises that use the server.

In addition to `backend-store-uri`, what else do you need to pass to properly configure the server?
 

In [11]:
!mlflow server --help

Usage: mlflow server [OPTIONS]

  Run the MLflow tracking server.

  The server listens on http://localhost:5000 by default and only accepts
  connections from the local machine. To let the server accept connections
  from other machines, you will need to pass ``--host 0.0.0.0`` to listen on
  all network interfaces (or a specific interface address).

Options:
  --backend-store-uri PATH        URI to which to persist experiment and run
                                  data. Acceptable URIs are SQLAlchemy-
                                  compatible database connection strings (e.g.
                                  'sqlite:///path/to/file.db') or local
                                  filesystem URIs (e.g.
                                  'file:///absolute/path/to/directory'). By
                                  default, data will be logged to the ./mlruns
                                  directory.
  --registry-store-uri URI        URI to which to persist registered models.
    

 --default-artifact-root

## Q5. Tune model hyperparameters
Now let's try to reduce the validation error by tuning the hyperparameters of the `RandomForestRegressor` using `hyperopt`. We have prepared the script `hpo.py` for this exercise.

Your task is to modify the script `hpo.py` and make sure that the validation RMSE is logged to the tracking server for each run of the hyperparameter optimization (you will need to add a few lines of code to the `objective` function) and run the script without passing any parameters.

After that, open UI and explore the runs from the experiment called `random-forest-hyperopt` to answer the question below.

Note: Don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

the list of hyperparameters that are passed to the `objective` function during the optimization,
the RMSE obtained on the validation set (February 2023 data).

In [13]:
!python hpo.py


  0%|          | 0/15 [00:00<?, ?trial/s, best loss=?]
                                                      
🏃 View run overjoyed-squid-808 at: http://127.0.0.1:5000/#/experiments/668922372659723857/runs/4bfaf4be02b7465e8ea8096a3ccc590c


  0%|          | 0/15 [00:33<?, ?trial/s, best loss=?]
                                                      
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/668922372659723857


  7%|▋         | 1/15 [00:33<07:54, 33.86s/trial, best loss: 5.370086069268862]
                                                                               
🏃 View run invincible-ant-547 at: http://127.0.0.1:5000/#/experiments/668922372659723857/runs/2c566f6a42ca45408b2465c0a9b2c734


  7%|▋         | 1/15 [00:35<07:54, 33.86s/trial, best loss: 5.370086069268862]
                                                                               
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/668922372659723857


 13%|█▎        | 2/15 [00:35<03:16, 15.11s


5.335

In [14]:
client.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/668922372659723857', creation_time=1748006219205, experiment_id='668922372659723857', last_update_time=1748006219205, lifecycle_stage='active', name='random-forest-hyperopt', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/510162088933463138', creation_time=1748003263023, experiment_id='510162088933463138', last_update_time=1748003263023, lifecycle_stage='active', name='homework2', tags={}>]

In [16]:
ordered_run = client.search_runs(
    experiment_ids=668922372659723857,
    filter_string='',
    max_results=1,
    order_by=['metrics.rmse ASC'])

ordered_run

[<Run: data=<RunData: metrics={'rmse': 5.335419588556921}, params={'max_depth': '19',
  'min_samples_leaf': '2',
  'min_samples_split': '2',
  'n_estimators': '11',
  'random_state': '42'}, tags={'mlflow.runName': 'upset-gnu-475',
  'mlflow.source.git.commit': '1258b40e09df1bea816b0ec8a58c2e0db01bfa41',
  'mlflow.source.name': 'hpo.py',
  'mlflow.source.type': 'LOCAL',
  'mlflow.user': 'crab'}>, info=<RunInfo: artifact_uri='mlflow-artifacts:/668922372659723857/a11548cecd92456d81f6ef388bc6e5f4/artifacts', end_time=1748006542695, experiment_id='668922372659723857', lifecycle_stage='active', run_id='a11548cecd92456d81f6ef388bc6e5f4', run_name='upset-gnu-475', run_uuid='a11548cecd92456d81f6ef388bc6e5f4', start_time=1748006528055, status='FINISHED', user_id='crab'>, inputs=<RunInputs: dataset_inputs=[]>>]

In [17]:
ordered_run_id = 'a11548cecd92456d81f6ef388bc6e5f4'

order_run = client.get_run(run_id=ordered_run_id)

In [18]:
order_run.data.metrics['rmse']

5.335419588556921

What's the best validation RMSE that you got?

**5.335**

## Q6. Promote the best model to the model registry
The results from the hyperparameter optimization are quite good. So, we can assume that we are ready to test some of these models in production. In this exercise, you'll promote the best model to the model registry. We have prepared a script called `register_model.py`, which will check the results from the previous step and select the top 5 runs. After that, it will calculate the RMSE of those models on the test set (March 2023 data) and save the results to a new experiment called `random-forest-best-models`.

Your task is to update the script `register_model.py` so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tip 1: you can use the method `search_runs` from the `MlflowClient` to get the model with the lowest RMSE,

Tip 2: to register the model you can use the method `mlflow.register_model` and you will need to pass the right `model_uri` in the form of a string that looks like this: `"runs:/<RUN_ID>/model"`, and the name of the model (make sure to choose a good one!).

In [19]:
!python register_model.py

🏃 View run popular-bat-213 at: http://127.0.0.1:5000/#/experiments/924568691926401311/runs/a070f661a9e54d6689ce3cd27487c7e0

2025/05/23 21:30:04 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-best-models' does not exist. Creating a new experiment.



🧪 View experiment at: http://127.0.0.1:5000/#/experiments/924568691926401311
🏃 View run mysterious-gnu-621 at: http://127.0.0.1:5000/#/experiments/924568691926401311/runs/26e37ec3f08f4f109caadd163dcd5b81
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/924568691926401311
🏃 View run inquisitive-gull-279 at: http://127.0.0.1:5000/#/experiments/924568691926401311/runs/a58e8956e9d74e3da677ba1c5279e32b
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/924568691926401311
🏃 View run gifted-calf-173 at: http://127.0.0.1:5000/#/experiments/924568691926401311/runs/99fc11dea25d4d5d9dcf451912d4b1f1
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/924568691926401311
🏃 View run judicious-snake-651 at: http://127.0.0.1:5000/#/experiments/924568691926401311/runs/f149a26679d54c5aa9ac1f3d5d2df35c
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/924568691926401311


In [23]:
client.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/924568691926401311', creation_time=1748007004266, experiment_id='924568691926401311', last_update_time=1748007004266, lifecycle_stage='active', name='random-forest-best-models', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/668922372659723857', creation_time=1748006219205, experiment_id='668922372659723857', last_update_time=1748006219205, lifecycle_stage='active', name='random-forest-hyperopt', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/510162088933463138', creation_time=1748003263023, experiment_id='510162088933463138', last_update_time=1748003263023, lifecycle_stage='active', name='homework2', tags={}>]

In [24]:
ordered_run = client.search_runs(
    experiment_ids=924568691926401311,
    filter_string='',
    max_results=1,
    order_by=['metrics.rmse ASC'])

ordered_run

[<Run: data=<RunData: metrics={'test_rmse': 5.5941605655803635,
  'training_mean_absolute_error': 3.323916924052877,
  'training_mean_squared_error': 26.08294493276463,
  'training_r2_score': 0.6796805248104354,
  'training_root_mean_squared_error': 5.107146456952711,
  'training_score': 0.6796805248104354,
  'val_rmse': 5.3633599989832135}, params={'bootstrap': 'True',
  'ccp_alpha': '0.0',
  'criterion': 'squared_error',
  'max_depth': '14',
  'max_features': '1.0',
  'max_leaf_nodes': 'None',
  'max_samples': 'None',
  'min_impurity_decrease': '0.0',
  'min_samples_leaf': '2',
  'min_samples_split': '6',
  'min_weight_fraction_leaf': '0.0',
  'monotonic_cst': 'None',
  'n_estimators': '23',
  'n_jobs': 'None',
  'oob_score': 'False',
  'random_state': '42',
  'verbose': '0',
  'warm_start': 'False'}, tags={'estimator_class': 'sklearn.ensemble._forest.RandomForestRegressor',
  'estimator_name': 'RandomForestRegressor',
  'mlflow.log-model.history': '[{"run_id": "f149a26679d54c5aa9ac1

In [25]:
ordered_run_id = 'f149a26679d54c5aa9ac1f3d5d2df35c'

order_run = client.get_run(run_id=ordered_run_id)

In [27]:
order_run.data.metrics['test_rmse']

5.5941605655803635


5.567